# Module 08: Multi-Agent Systems - Follow Along

**Key Topics:** Supervisor-worker, Team topologies, Agent communication, Production patterns

Run multi-agent examples from theory.


In [ ]:
%pip install -q -U langgraph

from langgraph.types import Command
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal

print('✅ Multi-agent tools ready!')

## Example 1: Supervisor-Worker Pattern


In [ ]:
class TeamState(TypedDict):
    task: str
    assigned_to: str
    result: str

def supervisor(state: TeamState) -> Command:
    """Supervisor assigns tasks to workers."""
    task = state['task'].lower()
    
    if 'research' in task:
        return Command(update={'assigned_to': 'researcher'}, goto='researcher')
    elif 'code' in task:
        return Command(update={'assigned_to': 'coder'}, goto='coder')
    return Command(update={'assigned_to': 'general'}, goto='general_worker')

def researcher(state: TeamState) -> Command:
    return Command(
        update={'result': f"Research complete: {state['task']}"},
        goto=END
    )

def coder(state: TeamState) -> Command:
    return Command(
        update={'result': f"Code written: {state['task']}"},
        goto=END
    )

def general_worker(state: TeamState) -> Command:
    return Command(
        update={'result': f"Task completed: {state['task']}"},
        goto=END
    )

# Build team
workflow = StateGraph(TeamState)
workflow.add_node('supervisor', supervisor)
workflow.add_node('researcher', researcher)
workflow.add_node('coder', coder)
workflow.add_node('general_worker', general_worker)
workflow.add_edge(START, 'supervisor')

app = workflow.compile()

# Test different tasks
tasks = [
    'Research Python async patterns',
    'Code a new feature',
    'Review documentation'
]

for task in tasks:
    result = app.invoke({'task': task, 'assigned_to': '', 'result': ''})
    print(f"Task: {task}")
    print(f"  Assigned to: {result['assigned_to']}")
    print(f"  Result: {result['result']}\n")

print('✅ Supervisor-worker delegation!')

## Example 2: Agent Handoffs


In [ ]:
class HandoffState(TypedDict):
    request: str
    tier: Literal['L1', 'L2', 'L3']
    resolution: str
    escalated: bool

def l1_support(state: HandoffState) -> Command:
    """Level 1 support - simple issues."""
    if 'password' in state['request'].lower():
        return Command(
            update={'resolution': 'Password reset link sent', 'tier': 'L1'},
            goto=END
        )
    # Escalate complex issues
    return Command(
        update={'tier': 'L2', 'escalated': True},
        goto='l2_support'
    )

def l2_support(state: HandoffState) -> Command:
    """Level 2 support - technical issues."""
    if 'bug' in state['request'].lower():
        return Command(
            update={'resolution': 'Bug report created', 'tier': 'L2'},
            goto=END
        )
    # Escalate to engineering
    return Command(
        update={'tier': 'L3', 'escalated': True},
        goto='l3_engineering'
    )

def l3_engineering(state: HandoffState) -> Command:
    return Command(
        update={'resolution': 'Engineering investigation started', 'tier': 'L3'},
        goto=END
    )

# Build escalation chain
workflow = StateGraph(HandoffState)
workflow.add_node('l1_support', l1_support)
workflow.add_node('l2_support', l2_support)
workflow.add_node('l3_engineering', l3_engineering)
workflow.add_edge(START, 'l1_support')

app = workflow.compile()

# Test escalation
requests = [
    'Forgot my password',
    'Found a bug in the app',
    'System architecture issue'
]

for req in requests:
    result = app.invoke({'request': req, 'tier': 'L1', 'resolution': '', 'escalated': False})
    print(f"Request: {req}")
    print(f"  Final tier: {result['tier']}")
    print(f"  Resolution: {result['resolution']}\n")

print('✅ Multi-tier agent handoffs!')

## Example 3: Team Topologies


In [ ]:
# Sequential team (pipeline)
print("Team Topology 1: Sequential (Pipeline)")
print("  Design → Implement → Test → Deploy")
print("  Use: Waterfall workflows\n")

# Hierarchical team (supervisor)
print("Team Topology 2: Hierarchical (Supervisor)")
print("  Manager → [Worker1, Worker2, Worker3]")
print("  Use: Task distribution, load balancing\n")

# Network team (peer-to-peer)
print("Team Topology 3: Network (Peer-to-Peer)")
print("  Agent1 ⟷ Agent2 ⟷ Agent3 ⟷ Agent4")
print("  Use: Collaborative problem solving\n")

# Hybrid team
print("Team Topology 4: Hybrid")
print("  Supervisor → [Sequential Team, Parallel Team]")
print("  Use: Complex enterprise workflows\n")

print('✅ Different topologies for different needs!')

## Example 4: Agent Communication


In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages

class SharedState(TypedDict):
    messages: Annotated[list, add_messages]
    shared_context: dict
    current_agent: str

def agent1(state: SharedState) -> Command:
    # Agent 1 adds to shared context
    context_update = {'findings': 'Data analysis complete'}
    return Command(
        update={
            'messages': [{'role': 'agent1', 'content': 'Analysis done'}],
            'shared_context': {**state.get('shared_context', {}), **context_update},
            'current_agent': 'agent1'
        },
        goto='agent2'
    )

def agent2(state: SharedState) -> Command:
    # Agent 2 uses shared context from Agent 1
    findings = state['shared_context'].get('findings', 'None')
    return Command(
        update={
            'messages': [{'role': 'agent2', 'content': f'Using: {findings}'}],
            'current_agent': 'agent2'
        },
        goto=END
    )

# Build communication flow
workflow = StateGraph(SharedState)
workflow.add_node('agent1', agent1)
workflow.add_node('agent2', agent2)
workflow.add_edge(START, 'agent1')

app = workflow.compile()
result = app.invoke({'messages': [], 'shared_context': {}, 'current_agent': ''})

print("Agent Communication:")
for msg in result['messages']:
    print(f"  {msg['role']}: {msg['content']}")
print(f"\nShared context: {result['shared_context']}")
print('✅ Agents share state and context!')

## Summary

You've run examples for:
- ✅ Supervisor-worker pattern
- ✅ Agent handoffs and escalation
- ✅ Team topologies (sequential, hierarchical, network)
- ✅ Agent communication via shared state

**Next:** `module-08-practice.ipynb` for multi-agent exercises!

## 🎉 Course Complete!

You've completed all 8 follow-along notebooks. You now have:
- ✅ **Theory**: 8 comprehensive markdown documents
- ✅ **Follow-Along**: 8 executable example notebooks
- ✅ **Practice**: 8 hands-on exercise notebooks

**Total:** 24 learning resources for mastering LangGraph! 🚀
